# Reinforcement Learning from Human Feedback

As the next chapter of our "Transformer for Summarization" series, we will delve into the application of reinforcement learning from human feedback (RLHF) for natural language processing tasks, as introduced in the InstructGPT paper (https://arxiv.org/pdf/2203.02155.pdf). Building on the foundations laid in our previous assignments, we will implement the RLHF algorithm to tackle the news summarization task. Here are the steps of this algorithms:

1. Generate multiple hypotheses for each input example using a small unlabeled training set.
2. Obtain human feedback by having "humans" rank these hypotheses according to their quality.
3. Train a reward model that captures human preferences by fitting it to the collected human feedback.
4. Leverage the reward model to enhance our summarization model using reinforcement learning techniques.

**Note:** Ensure you run this notebook on a CUDA GPU to optimize training speed. For instance, you can use a GPU instance on Google Colab.

In [1]:
#@title Install Packages

!python -m pip install datasets==2.11.0 transformers==4.16.2 tokenizers==0.13.2 evaluate==0.4.0 rouge_score==0.1.2 einops==0.6.0

%load_ext autoreload
%autoreload 2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB

In [2]:
# @title Import Packages

import os
import base64
import functools
import gzip
import json
import time
from tqdm.auto import tqdm
import random

import numpy as np
import evaluate

import einops
import torch
from datasets import load_dataset
from torch import nn
from torch.nn import functional as F
from torch.utils.data import RandomSampler
from datasets import Dataset
from transformers import (
    PreTrainedTokenizerBase,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    DataCollatorWithPadding,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    T5ForConditionalGeneration,
    RobertaPreTrainedModel,
    RobertaModel,
    TrainingArguments,
    Trainer,
)
from transformers.modeling_outputs import ModelOutput
from transformers.data.data_collator import PaddingStrategy
from typing import Optional, Union, Tuple, Any

from dataclasses import dataclass

N_HYPOTHESIS = 4
N_HYPOTHESIS_PAIRS = N_HYPOTHESIS * (N_HYPOTHESIS - 1) // 2  # C(N, 2)


def _set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)


_set_seed(1998)

TO_SAVE = {"time": time.time()}

In [3]:
# @title Test Utilities

def vector_all_close(x, y, rtol=1e-3):
    return ((x - y).norm(p=2, dim=-1) <= rtol * (x.norm(p=2, dim=-1) + y.norm(p=2, dim=-1))).all().item()

In [4]:
#@title Load Data

xsum = load_dataset("xsum")
blob = b'...'
labeled_train_indices, rank_train_indices, unlabeled_train_indices, validation_indices, test_indices = json.loads(
    gzip.decompress(base64.b64decode(blob)).decode("utf-8")
)
dataset_labeled_train = xsum["train"].select(labeled_train_indices)
dataset_rank_train = xsum["train"].select(rank_train_indices)
dataset_unlabeled_train = xsum["train"].select(unlabeled_train_indices)
dataset_validation = xsum["validation"].select(validation_indices)
dataset_test = xsum["test"].select(test_indices)

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Our dataset has been divided into five distinct splits: a **labeled** training set, an **unlabeled** training set for human ranking, an **unlabeled** training set for reinforcement learning, a validation set, and a test set. Note that we do not have access to the ground truth outputs for the unlabeled datasets.

In [5]:
dataset_labeled_train, dataset_rank_train, dataset_unlabeled_train, dataset_validation, dataset_test

(Dataset({
     features: ['document', 'summary', 'id'],
     num_rows: 2469
 }),
 Dataset({
     features: ['document', 'summary', 'id'],
     num_rows: 6562
 }),
 Dataset({
     features: ['document', 'summary', 'id'],
     num_rows: 15319
 }),
 Dataset({
     features: ['document', 'summary', 'id'],
     num_rows: 1281
 }),
 Dataset({
     features: ['document', 'summary', 'id'],
     num_rows: 1326
 }))

## Supervised Fine-tuning

The first step in the RLHF algorithm is supervised fine-tuning. We have already fine-tuned a T5-Small model on the small labeled training dataset for you, as covered in Part II of the assignment. To save time, please download the pre-trained model from [this link](https://drive.google.com/file/d/1CEeGY4j9Cecovv2ug0q3O5p2TAH2XNoH/view?usp=share_link) and place it in `/content/drive/MyDrive/cs182hw12_rlhf/`. Next, mount your Google Drive using the provided code.

In [6]:
from google.colab import drive, files
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Thanks to Sandeep Mukherjee for the utility script to download files
!gdown https://drive.google.com/file/d/1CEeGY4j9Cecovv2ug0q3O5p2TAH2XNoH/view?usp=share_link --fuzzy
!mkdir -p /content/drive/MyDrive/cs182hw12_rlhf
!mv /content/t5_small_xsum.bin /content/drive/MyDrive/cs182hw12_rlhf

Downloading...
From: https://drive.google.com/uc?id=1CEeGY4j9Cecovv2ug0q3O5p2TAH2XNoH
To: /content/t5_small_xsum.bin
100% 242M/242M [00:00<00:00, 316MB/s]


In [7]:
assert os.path.isfile("/content/drive/MyDrive/cs182hw12_rlhf/t5_small_xsum.bin")

In [8]:
# @title Process Data for Fine-tuning

tokenizer = AutoTokenizer.from_pretrained("t5-small")


def tokenize(tokenizer, examples, remove_target=False):
    inputs = [doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    if not remove_target:
        labels = tokenizer(examples["summary"], max_length=128, truncation=True)
        model_inputs["labels"] = labels["input_ids"]
    return model_inputs


tokenize_fn = functools.partial(tokenize, tokenizer, remove_target=False)
tokenize_fn_no_target = functools.partial(tokenize, tokenizer, remove_target=True)

dataset_labeled_train = dataset_labeled_train.map(tokenize_fn, batched=True)
dataset_rank_train = dataset_rank_train.map(tokenize_fn_no_target, batched=True)
dataset_unlabeled_train = dataset_unlabeled_train.map(tokenize_fn_no_target, batched=True)
dataset_validation = dataset_validation.map(tokenize_fn, batched=True)
dataset_test = dataset_test.map(tokenize_fn, batched=True)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="t5-small")

Downloading:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Map:   0%|          | 0/2469 [00:00<?, ? examples/s]

Map:   0%|          | 0/6562 [00:00<?, ? examples/s]

Map:   0%|          | 0/15319 [00:00<?, ? examples/s]

Map:   0%|          | 0/1281 [00:00<?, ? examples/s]

Map:   0%|          | 0/1326 [00:00<?, ? examples/s]

In [9]:
# @title Build Metrics

rouge = evaluate.load("rouge")


def summarization_metrics(tokenizer, eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}


summarization_metrics_fn = functools.partial(summarization_metrics, tokenizer)

In [10]:
# @title Evaluate the Finetuned Model

model = T5ForConditionalGeneration.from_pretrained("t5-small")
model.config.max_length = 64

model.load_state_dict(torch.load("/content/drive/MyDrive/cs182hw12_rlhf/t5_small_xsum.bin", map_location="cpu"))

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/cs182hw12_rlhf/t5_small_xsum_outs",
    learning_rate=1e-4,
    weight_decay=0.01,
    warmup_ratio=0.1,

    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    dataloader_drop_last=True,
    per_device_eval_batch_size=16,
    max_steps=2000,

    predict_with_generate=True,
    generation_num_beams=4,
    push_to_hub=False,
    logging_dir="/content/drive/MyDrive/cs182hw12_rlhf/t5_small_xsum_outs/logs",
    logging_strategy="steps",
    logging_first_step=True,
    logging_steps=1,
    save_strategy="steps",
    save_steps=500,
    evaluation_strategy="no",
    fp16=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset_labeled_train,
    eval_dataset=dataset_validation,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=summarization_metrics_fn
)

trainer.evaluate()

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

max_steps is given, it will override any value given in num_train_epochs
Using amp half precision backend
The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: document, summary, id.
***** Running Evaluation *****
  Num examples = 1281
  Batch size = 16


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-a105f6c117ec>", line 43, in <cell line: 43>
    trainer.evaluate()
  File "/usr/local/lib/python3.9/dist-packages/transformers/trainer_seq2seq.py", line 70, in evaluate
    return super().evaluate(eval_dataset, ignore_keys=ignore_keys, metric_key_prefix=metric_key_prefix)
  File "/usr/local/lib/python3.9/dist-packages/transformers/trainer.py", line 2208, in evaluate
    output = eval_loop(
  File "/usr/local/lib/python3.9/dist-packages/transformers/trainer.py", line 2454, in evaluation_loop
    metrics = self.compute_metrics(EvalPrediction(predictions=all_preds, label_ids=all_labels))
  File "<ipython-input-9-52dcd7e2b399>", line 12, in summarization_metrics
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  File "/usr/l

TypeError: ignored

## Collect Human Feedback

In the second step of the RLHF algorithm, your model will generate multiple summaries for each input sequence in a smaller unlabeled dataset. The model will run *random sampling* instead of *beam search* in this step to encourage diversified generations. These summaries will then be ranked by humans according to their quality. Humans can efficiently rank summaries, so we will ask them to order the generated summaries from best to worst for each example. However, since we don't have access to human data labelers in this assignment, we will use the ROUGE score computed with the ground truth targets to determine the ranks. Keep in mind that in real-world tasks, you wouldn't have access to these ground truth targets!

In this assignment, we generate 4 summaries for each example.

In [ ]:
def generate_dataset_for_ranking(trainer, dataset):
    trainer.model = trainer._wrap_model(trainer.model, training=False)
    trainer.args.dataloader_drop_last = False
    output_dataset = [
        {
            "id": item["id"],
            "document": item["document"],
            "hypothesis": []
        }
        for item in dataset
    ]
    for choice_id in range(N_HYPOTHESIS):
        generated_hypothesis = []
        dataloader = trainer.get_test_dataloader(dataset)
        for inputs in tqdm(dataloader):
            inputs = trainer._prepare_inputs(inputs)
            generation_inputs = inputs[trainer.model.main_input_name]
            generated_tokens = trainer.model.generate(
                generation_inputs,
                attention_mask=inputs.get("attention_mask", None),
                num_beams=1,
                do_sample=True  # this is how to do random sampling
            )
            generated_hypothesis.extend(
                tokenizer.batch_decode(
                    generated_tokens,
                    skip_special_tokens=True,
                    clean_up_tokenization_spaces=True
                )
            )
        assert len(generated_hypothesis) == len(dataset)
        for i in range(len(generated_hypothesis)):
            output_dataset[i]["hypothesis"].append(generated_hypothesis[i])
    return output_dataset

_set_seed(100)
dataset_for_ranking_train = generate_dataset_for_ranking(trainer, dataset_rank_train)

_set_seed(200)
dataset_for_ranking_validation = generate_dataset_for_ranking(trainer, dataset_validation)

The following columns in the test set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: id, document, summary.


  0%|          | 0/411 [00:00<?, ?it/s]

The following columns in the test set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: id, document, summary.


  0%|          | 0/411 [00:00<?, ?it/s]

The following columns in the test set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: id, document, summary.


...

  0%|          | 0/81 [00:00<?, ?it/s]

The following columns in the test set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: id, document, summary.


  0%|          | 0/81 [00:00<?, ?it/s]

The following columns in the test set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: id, document, summary.


  0%|          | 0/81 [00:00<?, ?it/s]

In [ ]:
def rank_with_human_labeler(dataset_orig, dataset_for_ranking):
    assert len(dataset_orig) == len(dataset_for_ranking)
    result = []
    for item0, item1 in tqdm(zip(dataset_orig, dataset_for_ranking), total=len(dataset_orig)):
        assert item0['document'] == item1['document']
        scores = []
        # We use a heuristic combination of ROUGE scores with some randomness to
        # simulate a human labeler.
        # Note that you cannot compute ROUGE scores in real-world applications
        # because you don't have groundtruth targets!
        w1 = 0.65 + np.random.rand() * 0.1  # 0.65 to 0.85
        w2 = 0.15 * np.random.rand() * 0.1  # 0.15 to 0.25
        w3 = 1.0 - w1 - w2
        for i in range(4):
            m = rouge.compute(
                predictions=[item0['summary']], references=[item1['hypothesis'][i]], use_stemmer=True
            )
            scores.append(m['rouge2'] * w1 + m['rougeL'] * w2 + m['rouge1'] * w3 + np.random.rand() * 0.001)
        item2 = item1.copy()
        item2['hypothesis'] = [item1['hypothesis'][j] for j in np.argsort(scores)[::-1]]
        result.append(item2)
    return result

_set_seed(300)
dataset_for_ranking_train = rank_with_human_labeler(dataset_rank_train, dataset_for_ranking_train)

_set_seed(400)
dataset_for_ranking_validation = rank_with_human_labeler(dataset_validation, dataset_for_ranking_validation)

  0%|          | 0/6562 [00:00<?, ?it/s]

  0%|          | 0/1281 [00:00<?, ?it/s]

Since the ranking simulator can be slow, we will save the outputs to your Google Drive. This way, you won't have to rerun this section every time you restart your Colab runtime. **You can also skip this step by downloading the files from this link.**

In [ ]:
with open("/content/drive/MyDrive/cs182hw12_rlhf/dataset_for_ranking_train.jsonl", "w", encoding="utf-8") as f:
    for item in dataset_for_ranking_train:
        f.write(json.dumps(item) + "\n")

with open("/content/drive/MyDrive/cs182hw12_rlhf/dataset_for_ranking_validation.jsonl", "w", encoding="utf-8") as f:
    for item in dataset_for_ranking_validation:
        f.write(json.dumps(item) + "\n")

## Train a Reward Model

The third step of the RLHF algorithm involves training a reward model. This model will be fitted to the human feedback and used to calculate rewards for reinforcement learning.

For each example, the reward model takes four sentence pairs as input, with each pair consisting of the document and one of the generated summaries. The reward model is a Transformer encoder like BERT, so it outputs a score for each sentence pair using its regression head over the CLS token. We train the model using this loss function:

$$
\mathbb{E}_{\mathbf{x}\sim D}[-\dfrac{1}{\binom{K}{2}} \sum_{1 \le i < j \le K} \log \sigma (r_\theta(x_i) - r_\theta(x_j))]
$$

Here, $K=4$ is the number of candidates. For each data example $(x_1, x_2, \dots, x_K)$ sampled from the dataset $D$, we have $x_1, x_2, \dots, x_K$ ranked from best to worst. The output of the reward model is $r_\theta(x_1), r_\theta(x_2),\dots,r_\theta(x_k)$. For each pair of $i < j$, we know $x_i$ is better than $x_j$, so we maximize $r_\theta(x_i)- r_\theta(x_j)$. The elementwise sigmoid function, denoted as $\sigma$, is monotonically increasing with its input. The loss is averaged over each example in the dataset.

Please **implement this loss function** , which will be used in the definition of `RobertaForRanking`. Make sure your implementation pass the test.

*Hint: The implementation can be fully vectorized (without any for-loops). You may find `torch.triu` useful for this purpose.*

*Hint: Use `torch.nn.functional.logsigmoid` instead two functions for better numerical stability.*

In [11]:
def ranking_loss(grouped_rewards):
    """
    grouped_rewards is a float tensor of shape [n_examples, N_HYPOTHESIS]
    where N_HYPOTHESIS = 4
    """
    ############################################################################
    # TODO: Implement this loss function
    ############################################################################
    reward_diff = grouped_rewards[:, :, None] - grouped_rewards[:, None, :]
    lprobs_triu = torch.triu(F.logsigmoid(reward_diff), diagonal=1)
    lprobs = einops.reduce(lprobs_triu, "b c1 c2 -> b", "sum") / N_HYPOTHESIS_PAIRS
    loss = -lprobs.mean()
    ############################################################################
    return loss

In [12]:
def test_ranking_loss():
    _set_seed(500)
    grouped_rewards = torch.rand(3, 4, requires_grad=True)
    loss = ranking_loss(grouped_rewards)
    assert torch.allclose(loss, torch.tensor(0.6951866745948792), rtol=1e-3)
    loss.backward()
    assert vector_all_close(
        grouped_rewards.grad.view(-1)[2:7],
        torch.tensor([0.046410925686359406, 0.0752371996641159, -0.07664521038532257, -0.023392682895064354, 0.0075188204646110535]),
        rtol=1e-3
    )

    _set_seed(550)
    grouped_rewards = torch.rand(5, 4, requires_grad=True)
    loss = ranking_loss(grouped_rewards)
    TO_SAVE["ranking_loss.forward"] = loss.item()
    loss.backward()
    TO_SAVE["ranking_loss.backward"] = grouped_rewards.grad.view(-1).tolist()[3:8]

test_ranking_loss()

In [13]:
@dataclass
class RankingModelOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None

In [14]:
class RobertaForRanking(RobertaPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)

        self.roberta = RobertaModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.regressor = nn.Linear(config.hidden_size, 1)

        self.post_init()

    def forward(
        self, input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        inference_mode: bool = False
    ):
        assert self.config.use_return_dict

        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            return_dict=True
        )
        pooled_output = outputs.pooler_output

        pooled_output = self.dropout(pooled_output)
        rewards = self.regressor(pooled_output)

        loss = None
        if not inference_mode:
            grouped_rewards = rewards.view(-1, N_HYPOTHESIS).float()
            loss = ranking_loss(grouped_rewards)

        return RankingModelOutput(
            loss=loss,
            logits=rewards.view(-1, 1),
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions
        )

We will fine-tune a Distill-RoBERTa model as the reward model. RoBERTa is an enhanced version of BERT, which is a pre-trained Transformer encoder model suitable for sentence classification tasks. Distill-RoBERTa is a knowledge-distilled version of RoBERTa, offering faster processing and reduced GPU memory usage.

We've designed the `RobertaForRanking` model above to inherit from `RobertaPreTrainedModel`, allowing it to load any pre-trained RoBERTa-like model checkpoints available on the Huggingface Model Hub.

In [15]:
reward_model_tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
reward_model = RobertaForRanking.from_pretrained("distilroberta-base")

Could not locate the tokenizer configuration file, will try to use the model config instead.
https://huggingface.co/distilroberta-base/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpeu9pn2wg


Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

storing https://huggingface.co/distilroberta-base/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
creating metadata file for /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "_name_or_path": "distilroberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

storing https://huggingface.co/distilroberta-base/resolve/main/vocab.json in cache at /root/.cache/huggingface/transformers/23e0f7484fc8a320856b168861166b48c2976bb4e0861602422e1b0c3fe5bf61.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
creating metadata file for /root/.cache/huggingface/transformers/23e0f7484fc8a320856b168861166b48c2976bb4e0861602422e1b0c3fe5bf61.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
https://huggingface.co/distilroberta-base/resolve/main/merges.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp64615zg2


...

storing https://huggingface.co/distilroberta-base/resolve/main/merges.txt in cache at /root/.cache/huggingface/transformers/c7e8020011da613ff5a9175ddad64cd47238a9525db975eb50ecb965e9f7302f.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
creating metadata file for /root/.cache/huggingface/transformers/c7e8020011da613ff5a9175ddad64cd47238a9525db975eb50ecb965e9f7302f.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
https://huggingface.co/distilroberta-base/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpzp_zlkip


Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

storing https://huggingface.co/distilroberta-base/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/b6a9ca6504e67903474c3fdf82ba249882406e61c2176a9d4dc9c3691c663767.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730
creating metadata file for /root/.cache/huggingface/transformers/b6a9ca6504e67903474c3fdf82ba249882406e61c2176a9d4dc9c3691c663767.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730
loading file https://huggingface.co/distilroberta-base/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/23e0f7484fc8a320856b168861166b48c2976bb4e0861602422e1b0c3fe5bf61.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/distilroberta-base/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/c7e8020011da613ff5a9175ddad64cd47238a9525db975eb50ecb965e9f7302f.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://hugg

Downloading:   0%|          | 0.00/316M [00:00<?, ?B/s]

storing https://huggingface.co/distilroberta-base/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/7a0115a4c463f49bc7ab011872fc4a4b81be681a0434075955d29ac3388e225b.a6127d76576e81475313180aceb31a8688f7a649b80e380d26b5d30302dc83c1
creating metadata file for /root/.cache/huggingface/transformers/7a0115a4c463f49bc7ab011872fc4a4b81be681a0434075955d29ac3388e225b.a6127d76576e81475313180aceb31a8688f7a649b80e380d26b5d30302dc83c1
loading weights file https://huggingface.co/distilroberta-base/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/7a0115a4c463f49bc7ab011872fc4a4b81be681a0434075955d29ac3388e225b.a6127d76576e81475313180aceb31a8688f7a649b80e380d26b5d30302dc83c1
Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForRanking: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS exp

Now we will process the previously saved dataset for training the reward model. Note that Distill-RoBERTa uses a different tokenizer compared to T5-small.

The `tokenizer_ranking_example` function converts each example of `(document, hypothesis1, hypothesis2, hypothesis3, hypothesis4)` into four sentence pairs: `(document, hypothesis1)`, `(document, hypothesis2)`, `(document, hypothesis3)`, and `(document, hypothesis4)`, each of which is concatenated into one sequence. As a result, the tokenized example (called "features" in Huggingface `transformers`) count is four times the example count.

In [16]:
def tokenize_ranking_example(tokenizer, examples):
    all_documents = [doc for doc in examples["document"] for _ in range(4)]
    all_hypothesis = [hyp for hyps in examples["hypothesis"] for hyp in hyps]
    assert len(all_documents) == len(all_hypothesis)
    features = tokenizer(
        all_documents, all_hypothesis, max_length=512, truncation=True
    )
    return features

tokenize_ranking_fn = functools.partial(tokenize_ranking_example, reward_model_tokenizer)
dataset_for_ranking = load_dataset("json", data_files={
    "train": "/content/drive/MyDrive/cs182hw12_rlhf/dataset_for_ranking_train.jsonl",
    "validation": "/content/drive/MyDrive/cs182hw12_rlhf/dataset_for_ranking_validation.jsonl"
})
print(json.dumps(dataset_for_ranking["train"][0], indent=4))
dataset_for_ranking_train = dataset_for_ranking["train"].map(
    tokenize_ranking_fn, batched=True, remove_columns=dataset_for_ranking["train"].column_names
)
dataset_for_ranking_validation = dataset_for_ranking["validation"].map(
    tokenize_ranking_fn, batched=True, remove_columns=dataset_for_ranking["validation"].column_names
)

reward_model_data_collator = DataCollatorWithPadding(tokenizer=reward_model_tokenizer, pad_to_multiple_of=8)
dataset_for_ranking_train, dataset_for_ranking_validation

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-fab82772447593ac/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{
    "id": "35890902",
    "document": "Weaknesses in the way mice swapped data with computers left them vulnerable, said security firm Bastille Networks.\nAttackers could spoof poorly protected signals letting them use PCs as if they were sitting in front of them, it said.\nInformation about the loopholes have been passed to the makers of vulnerable mice, some of who are creating updates to make the mice more secure.\nThe radio signals sent by many wireless mice to a \"dongle\" plugged in to a computer were often unencrypted, said Marc Newlin and Balint Seeber, from Bastille, who carried out the research.\n\"That makes it possible for the attacker to send unencrypted traffic to the dongle pretending to be a keyboard and have it result as keystrokes on your computer,\" Mr Newlin said.\nBy contrast, they said, signals sent by wireless keyboards were scrambled to stop attackers eavesdropping on or spoofing them.\nThe pair found they could spoof signals for mice using a few lines of code

Map:   0%|          | 0/6562 [00:00<?, ? examples/s]

Map:   0%|          | 0/1281 [00:00<?, ? examples/s]

(Dataset({
     features: ['input_ids', 'attention_mask'],
     num_rows: 26248
 }),
 Dataset({
     features: ['input_ids', 'attention_mask'],
     num_rows: 5124
 }))

Our ranking loss implementation and model require that each batch has a number of sentence pairs that is a multiple of 4, with every 4 sentence pairs belonging to the same example grouped together. During training, however, the `Trainer` in the `transformers` library randomly shuffles all processed training examples in each epoch. We need to manage this shuffling behavior to maintain enough randomness while preserving the grouping of every 4 sentence pairs. To achieve this, we will define a `GroupedRandomSampler` as the data sampler for the dataloader in `Trainer`. Please **implement this `GroupedRandomSampler`**.

In [17]:
random_sampler = RandomSampler(np.arange(20) * 2)
print("RandomSampler indices:", list(random_sampler))


class GroupedRandomSampler:
    def __init__(self, data_source):
        self.data_source = data_source

    def __iter__(self):
        n = len(self.data_source)
        ########################################################################
        # TODO: Modify the following part of a RandomSampler to preserve groups
        #       of 4
        ########################################################################
        # yield from torch.randperm(n).tolist()
        for idx in torch.randperm(n // N_HYPOTHESIS).tolist():
            yield from torch.arange(idx * N_HYPOTHESIS, idx * N_HYPOTHESIS + N_HYPOTHESIS).tolist()
        ########################################################################

    def __len__(self):
        return len(self.data_source)


random_sampler = GroupedRandomSampler(np.arange(20) * 2)
print("GroupedRandomSampler indices:", list(random_sampler))


class RankingTrainer(Trainer):
    def _get_train_sampler(self):
        assert not self.args.group_by_length
        return GroupedRandomSampler(self.train_dataset)

RandomSampler indices: [8, 1, 14, 2, 11, 6, 18, 10, 15, 9, 3, 0, 13, 5, 4, 12, 7, 16, 17, 19]
GroupedRandomSampler indices: [0, 1, 2, 3, 12, 13, 14, 15, 16, 17, 18, 19, 4, 5, 6, 7, 8, 9, 10, 11]


In [18]:
def test_group_random_sampler():
    random_sampler = GroupedRandomSampler(np.arange(8) * 2)
    sampled_indices = list(random_sampler)
    assert (
        sampled_indices == [0, 1, 2, 3, 4, 5, 6, 7]
        or sampled_indices == [4, 5, 6, 7, 0, 1, 2, 3]
    )

    random_sampler = GroupedRandomSampler(np.arange(24) * 3)
    sampled_indices = list(random_sampler)
    TO_SAVE["group_random_sampler"] = sampled_indices

test_group_random_sampler()

The metric we use to evaluate the reward model is the Spearman Correlation coefficient. The Spearman Correlation coefficient measures the strength and direction of the monotonic relationship between two ranked variables. For instance, if the model's output ranking exactly matches the ground truth, the correlation coefficient is 1. Conversely, if the model's output ranking is the complete opposite of the ground truth, the correlation coefficient is -1.

In [19]:
def reward_model_metrics_fn(eval_preds):
    rewards, _ = eval_preds
    rewards = rewards.reshape(-1, N_HYPOTHESIS)
    reward_ranks = rewards.argsort(axis=-1)[:, ::-1].argsort(axis=-1)
    true_ranks = np.arange(N_HYPOTHESIS)
    reward_ranks = (reward_ranks - reward_ranks.mean(axis=-1, keepdims=True)) / reward_ranks.std(axis=-1, keepdims=True)
    true_ranks = (true_ranks - true_ranks.mean()) / true_ranks.std()
    correlation_coef = (reward_ranks * true_ranks).sum(axis=-1) / N_HYPOTHESIS
    return {"spearman_corr": correlation_coef.mean()}

Now, we will fine-tune the Distill-RoBERTa model on the ranking objective for 3 epochs using the dataset. The training process should take approximately **25 minutes**. If your implementation is correct, you can expect a Spearman Correlation of around **0.26** on the validation set after training.

In [20]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/cs182hw12_rlhf/rank_model_outs",
    learning_rate=1.5e-5,
    weight_decay=0.01,
    warmup_ratio=0.1,

    per_device_train_batch_size=32,
    gradient_accumulation_steps=2,
    dataloader_drop_last=True,
    label_names=["input_ids"],
    per_device_eval_batch_size=32,
    num_train_epochs=3,

    push_to_hub=False,
    logging_dir="/content/drive/MyDrive/cs182hw12_rlhf/rank_model_outs/logs",
    logging_strategy="steps",
    logging_first_step=True,
    logging_steps=1,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    fp16=True
)

trainer = RankingTrainer(
    model=reward_model,
    args=training_args,
    train_dataset=dataset_for_ranking_train,
    eval_dataset=dataset_for_ranking_validation,
    tokenizer=reward_model_tokenizer,
    data_collator=reward_model_data_collator,
    compute_metrics=reward_model_metrics_fn
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using amp half precision backend
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 26248
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 2
  Total optimization steps = 1230


Epoch,Training Loss,Validation Loss,Spearman Corr
1,0.653700,0.670223,0.215156
2,0.613500,0.659744,0.263437
3,0.629600,0.666295,0.265781


***** Running Evaluation *****
  Num examples = 5124
  Batch size = 32
Saving model checkpoint to /content/drive/MyDrive/cs182hw12_rlhf/rank_model_outs/checkpoint-410
Configuration saved in /content/drive/MyDrive/cs182hw12_rlhf/rank_model_outs/checkpoint-410/config.json
Model weights saved in /content/drive/MyDrive/cs182hw12_rlhf/rank_model_outs/checkpoint-410/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/cs182hw12_rlhf/rank_model_outs/checkpoint-410/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/cs182hw12_rlhf/rank_model_outs/checkpoint-410/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 5124
  Batch size = 32
Saving model checkpoint to /content/drive/MyDrive/cs182hw12_rlhf/rank_model_outs/checkpoint-820
Configuration saved in /content/drive/MyDrive/cs182hw12_rlhf/rank_model_outs/checkpoint-820/config.json
Model weights saved in /content/drive/MyDrive/cs182hw12_rlhf/rank_model_outs/checkpoint-820/pytorch_mode

TrainOutput(global_step=1230, training_loss=0.6553642255988548, metrics={'train_runtime': 1474.0812, 'train_samples_per_second': 53.419, 'train_steps_per_second': 0.834, 'total_flos': 1.0419434824341504e+16, 'train_loss': 0.6553642255988548, 'epoch': 3.0})

## Reinforcement Learning with the Reward Model



In [21]:
@dataclass
class DataCollatorForRLHF:

    tokenizer: PreTrainedTokenizerBase
    model: Optional[Any] = None
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    label_pad_token_id: int = -100
    return_tensors: str = "pt"

    def __call__(self, features, return_tensors=None):

        if return_tensors is None:
            return_tensors = self.return_tensors

        generation_features = [
            {
                "input_ids": feature["input_ids"],
                "attention_mask": feature["attention_mask"]
            }
            for feature in features
        ]
        generation_features = self.tokenizer.pad(
            generation_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=return_tensors
        )
        generation_features["document"] = [feature["document"] for feature in features]

        return generation_features

    def pad_labels(self, out_features, return_tensors=None):
        if return_tensors is None:
            return_tensors = self.return_tensors
        out_features.pop("attention_mask")

        max_label_length = max(len(l) for l in out_features["input_ids"])
        if self.pad_to_multiple_of is not None:
            max_label_length = (
                (max_label_length + self.pad_to_multiple_of - 1)
                // self.pad_to_multiple_of * self.pad_to_multiple_of
            )

        padding_side = self.tokenizer.padding_side
        for i in range(len(out_features["input_ids"])):
            remainder = [self.label_pad_token_id] * (max_label_length - len(out_features["input_ids"][i]))
            assert isinstance(out_features["input_ids"][i], list)
            out_features["input_ids"][i] = (
                out_features["input_ids"][i] + remainder
                if padding_side == "right"
                else remainder + out_features["input_ids"][i]
            )

        out_features = tokenizer.pad(
            out_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=return_tensors
        )

        out_features["labels"] = out_features["input_ids"]
        out_features.pop("input_ids")
        return out_features


data_collator_rlhf = DataCollatorForRLHF(tokenizer=tokenizer, model="t5-small")

The reinforcement learning training loop with rewards calculated by the reward model consists of the following steps for each input batch:

1. Generate one summary for each input document using the seq2seq model via random sampling.
2. Feed the document-summary pairs generated in step 1 to the reward model, obtaining the reward value for each pair.
3. Feed the document-summary pairs into the seq2seq model to get the log-probabilities of each pair.
  - Then, compute the loss function, which will be differentiated to obtain policy gradients. **You will implement this part as a standalone function `policy_gradient_loss`.**

In [49]:
def policy_gradient_loss(logits, generated, rewards, pad_token_id, label_pad_token_id=-100):
    """
    logits: FloatTensor[bsz, max_tgt_len, vocab_size]; the pre-softmax scores
    generated: LongTensor[bsz, max_tgt_len]; the tokens generated in step 1
    rewards: FloatTensor[bsz]; the rewards returned by the reward model
    pad_token_id: int; word index of the pad token
    label_pad_token_id: int; set to -100, which is the default padding id for
                             labels/generated tokens in `transformers`; it is
                             also the default `ignore_index` in PyTorch's
                             `nn.CrossEntropyLoss`
    """
    label_padding_mask = generated == label_pad_token_id
    logits[:, :, pad_token_id] = float("-inf")
    lprobs = torch.log_softmax(logits, dim=-1, dtype=torch.float32)

    ############################################################################
    # TODO: Complete the following implementation of `policy_gradient_loss`
    #       Note: one batch consists of generated outputs of different lengths,
    #             padded with `label_pad_token_id`, which can be negative
    #
    # Hint: Use `torch.gather` or `nn.CrossEntropyLoss`
    ############################################################################
    lprobs = lprobs.masked_fill(label_padding_mask.unsqueeze(dim=-1), 0.0)
    new_generated = generated.masked_fill(label_padding_mask, pad_token_id)
    label_lprobs = torch.gather(
        lprobs, dim=-1, index=new_generated.unsqueeze(-1)
    ).squeeze(dim=-1).sum(dim=-1)
    loss = -(label_lprobs * rewards).mean()
    ############################################################################
    
    return loss

In [59]:
def test_policy_gradient_loss():
    _set_seed(600)
    hidden_states = torch.randn(3, 6, 5, requires_grad=True)
    lm_head = nn.Linear(5, len(tokenizer))
    logits = lm_head(hidden_states)
    generated = torch.tensor([
        [27, 333, 25, 6, 1, -100],
        [3455, 23, 3464, 55, 1, -100],
        [27, 3041, 25, 78, 231, 1]
    ]).long()
    rewards = torch.rand(3)
    loss = policy_gradient_loss(logits, generated, rewards, tokenizer.pad_token_id)
    assert torch.allclose(loss, torch.tensor(18.744550704956055), rtol=1e-3)
    loss.backward()
    assert vector_all_close(
        hidden_states.grad.view(-1)[8:13],
        torch.tensor([-0.02723854035139084, -0.01294753048568964, 0.03945881873369217, 0.025191081687808037, 0.025536173954606056]),
        rtol=1e-3
    )

    _set_seed(650)
    hidden_states = torch.randn(4, 10, 5, requires_grad=True)
    lm_head = nn.Linear(5, len(tokenizer))
    logits = lm_head(hidden_states)
    generated = torch.tensor([
        [3, 4778, 2606, 357, 65, 1995, 13, 14023, 5, 1],
        [282, 3, 9, 3, 3221, 492, 175, 6, 1, -100],
        [27, 897, 25, 2525, 424, 5, 1, -100, -100, -100],
        [1804, 5851, 30, 39, 804, 7, 55, 1, -100, -100]
    ]).long()
    rewards = torch.rand(4)
    loss = policy_gradient_loss(logits, generated, rewards, tokenizer.pad_token_id)
    TO_SAVE["policy_gradient_loss.forward"] = loss.item()
    loss.backward()
    TO_SAVE["policy_gradient_loss.backward"] = hidden_states.grad.view(-1)[9:14].tolist()
  
test_policy_gradient_loss()

In the next code cell, we demonstrate the reinforcement learning training loop with rewards generated by a reward model. Ensure that the following code can run without runtime errors, and then stop after the first few steps.

In [60]:
class RLHFSeq2SeqTrainer(Seq2SeqTrainer):
    def prepare_reward_model(self, model, tokenizer):
        model.eval()
        model.half()
        model.to(self.args.device)
        self.reward_model = model
        self.reward_model_tokenizer = tokenizer

    def compute_loss(self, model, inputs, return_outputs=False):
        """
        The reinforcement learning training loop with rewards calculated by the
        reward model
        """

        model.eval()

        # Step 1: Generate a summary for each input document via random sampling
        with torch.no_grad():
            generated_tokens = model.generate(
                inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                num_beams=1,
                do_sample=True
            )
        generated_texts = self.tokenizer.batch_decode(
            generated_tokens,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True
        )

        # Step 2: Feed the document-summary pairs to the reward model,
        #         obtaining the reward value for each pair.
        rm_inputs = self.reward_model_tokenizer(
            inputs["document"],
            generated_texts,
            max_length=512,
            truncation=True
        )
        rm_inputs = self.reward_model_tokenizer.pad(
            rm_inputs,
            padding=True,
            return_tensors="pt"
        )
        rm_inputs = self._prepare_inputs(rm_inputs)
        with torch.no_grad():
            rewards = self.reward_model(
                **rm_inputs, inference_mode=True
            ).logits.view(-1)
            # "baseline" trick in policy gradient
            rewards = rewards - rewards.mean()

        model.train()

        # Step 3.1: Feed the document-summary pairs into the seq2seq model to
        #           get the log-probabilities of each pair.
        rl_inputs = self.tokenizer(
            generated_texts, max_length=1024, truncation=True
        )
        rl_inputs = self.tokenizer.pad(
            rl_inputs,
            padding=True,
            return_tensors="pt"
        )
        rl_outputs = self.tokenizer(
            generated_texts,
            max_length=128,
            truncation=True
        )
        rl_inputs["labels"] = self.data_collator.pad_labels(rl_outputs)["labels"]
        rl_inputs = self._prepare_inputs(rl_inputs)

        outputs = model(**rl_inputs)

        # Step 3.2: Compute the loss function, which will be differentiated to
        #           obtain policy gradients.
        loss = policy_gradient_loss(
            outputs.logits,
            rl_inputs['labels'],
            rewards.float(),
            self.tokenizer.pad_token_id
        )

        outputs.loss = loss
        return (loss, outputs) if return_outputs else loss

In [61]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/cs182hw12_rlhf/rlhf_model_outs",
    learning_rate=1e-4,
    weight_decay=0.01,
    warmup_ratio=0.1,

    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    dataloader_drop_last=True,
    per_device_eval_batch_size=16,
    max_steps=2000,
    remove_unused_columns=False,

    predict_with_generate=True,
    generation_num_beams=4,
    push_to_hub=False,
    logging_dir="/content/drive/MyDrive/cs182hw12_rlhf/rlhf_model_outs/logs",
    logging_strategy="steps",
    logging_first_step=True,
    logging_steps=1,
    save_strategy="steps",
    save_steps=500,
    evaluation_strategy="no",
    fp16=True
)

trainer = RLHFSeq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset_unlabeled_train,
    eval_dataset=dataset_validation,
    tokenizer=tokenizer,
    data_collator=data_collator_rlhf,
    compute_metrics=summarization_metrics_fn
)

trainer.prepare_reward_model(reward_model, reward_model_tokenizer)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
max_steps is given, it will override any value given in num_train_epochs
Using amp half precision backend
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 15319
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 2000


Step,Training Loss
1,0.842400
2,1.013000
3,1.887000
4,-1.644300
5,0.622200
6,0.647700
7,-0.412900
8,-0.701900
9,0.378500


KeyboardInterrupt: ignored

## Submission

Unfortunately, RLHF takes too much time to run as a coding assignment, so we will stop here. It's also likely that this minimal demo would not significantly improve the performance of our summarization model due to several reasons:

- The baseline fine-tuned model is weak and not a suitable starting point to observe the bootstrapping effect.
- The reward model is small, and more importantly, it only supports input sequences of fewer than 512 tokens. This is the only model we found that fits into the Colab GPU memory alongside T5-small. In InstructGPT, they use a reward model with 7B parameters, which is 117x larger.
- The ranking data is too limited, causing even the Distill-RoBERTa model to overfit within just 3 epochs.
- We use a simple policy gradient algorithm for RL, while InstructGPT employs Proximal Policy Optimization (PPO) jointly trained with supervised fine-tuning.
- The batch size for RL should typically be much larger than for supervised learning, but we have limited GPU memory available.

As you can see, the success of RLHF heavily relies on scale, encompassing data, model, and compute resources. Unlike other coding assignments (e.g., Part II of the summarization series), which effectively shows how NLP works in real life, this coding assignment only demonstrates the basic concepts of RLHF and is far from being able to work in practice.

You are done with this coding assignment. Please download submission_log.json and submit it to Gradescope.

In [63]:
with open("submission_logs.json", "w", encoding="utf-8") as f:
    json.dump(TO_SAVE, f)